<a href="https://colab.research.google.com/github/PETEROA/AutoML/blob/main/Demo_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Create Gradio web interface
- Load all optimized models
- Allow image upload for inference
- Compare different model formats in real-time
- Show speed/accuracy tradeoffs
- Create shareable demo link



In [ ]:
!pip install torch torchvision gradio -q
!pip install onnxruntime pillow -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import onnxruntime as ort
import gradio as gr
import numpy as np
from PIL import Image
from pathlib import Path
import json
import time
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

# Setup paths
OUTPUT_DIR = Path('/content/drive/MyDrive/AutoML')
MODELS_DIR = OUTPUT_DIR / 'models'
DEPLOYMENT_DIR = OUTPUT_DIR / 'deployment'
ONNX_DIR = DEPLOYMENT_DIR / 'onnx'
RESULTS_DIR = OUTPUT_DIR / 'results'

print(f"Output directory: {OUTPUT_DIR}")

✓ All imports successful
Output directory: /content/drive/MyDrive/AutoML


In [ ]:
# Load Deployment Summary
# CIFAR-10 class names
CIFAR10_CLASSES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# Load deployment summary
summary_file = DEPLOYMENT_DIR / 'deployment_summary.json'

if summary_file.exists():
    with open(summary_file, 'r') as f:
        deployment_info = json.load(f)
    print("✓ Loaded deployment summary")
    print(f"  Available models: {len(deployment_info.get('models', []))}")
else:
    print("⚠ Deployment summary not found, creating demo configuration")
    deployment_info = {
        'models': [{
            'rank': 1,
            'architecture': 'resnet18',
            'files': {
                'onnx': 'deployment/onnx/rank1_resnet18.onnx'
            }
        }]
    }

⚠ Deployment summary not found, creating demo configuration


In [ ]:
# Model Loading Functions
class ModelWrapper:
    """Wrapper for different model formats"""

    def __init__(self, model_path, model_type='onnx'):
        self.model_path = model_path
        self.model_type = model_type
        self.model = None
        self.load_model()

    def load_model(self):
        """Load the model"""
        if self.model_type == 'onnx':
            if Path(self.model_path).exists():
                self.model = ort.InferenceSession(
                    self.model_path,
                    providers=['CPUExecutionProvider']
                )
                print(f"✓ Loaded ONNX model: {Path(self.model_path).name}")
            else:
                print(f"⚠ Model not found: {self.model_path}")

        elif self.model_type == 'pytorch':
            # Load PyTorch model
            # This is a placeholder - would need actual model architecture
            pass

    def predict(self, image: np.ndarray):
        """Run inference"""
        if self.model is None:
            return None, None

        start_time = time.time()

        if self.model_type == 'onnx':
            # ONNX inference
            input_name = self.model.get_inputs()[0].name
            outputs = self.model.run(None, {input_name: image})[0]

        inference_time = (time.time() - start_time) * 1000  # Convert to ms

        return outputs, inference_time

# Image preprocessing
def preprocess_image(image: Image.Image) -> np.ndarray:
    """Preprocess image for CIFAR-10 models"""
    # Resize to 32x32 (CIFAR-10 size)
    image = image.resize((32, 32))

    # Convert to tensor and normalize
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465),
            (0.2023, 0.1994, 0.2010)
        )
    ])

    tensor = transform(image)

    # Add batch dimension and convert to numpy
    numpy_array = tensor.unsqueeze(0).numpy()

    return numpy_array

def postprocess_outputs(outputs: np.ndarray) -> dict:
    """Convert model outputs to predictions"""
    # Get probabilities
    probabilities = np.exp(outputs) / np.sum(np.exp(outputs), axis=1, keepdims=True)

    # Get top 3 predictions
    top3_idx = np.argsort(probabilities[0])[::-1][:3]

    predictions = {}
    for idx in top3_idx:
        class_name = CIFAR10_CLASSES[idx]
        confidence = float(probabilities[0][idx])
        predictions[class_name] = confidence

    return predictions

print("✓ Model wrapper and preprocessing functions ready")

✓ Model wrapper and preprocessing functions ready


In [ ]:
# Load Available Models
# Load all available ONNX models
available_models = {}

print("Loading available models...")
print("="*60)

for model_info in deployment_info.get('models', []):
    rank = model_info['rank']
    arch = model_info['architecture']

    # Try to load ONNX model
    onnx_file = model_info['files'].get('onnx', '')
    onnx_path = OUTPUT_DIR / onnx_file

    if onnx_path.exists():
        model_name = f"Rank {rank} - {arch.upper()}"
        available_models[model_name] = ModelWrapper(str(onnx_path), 'onnx')
        print(f"✓ {model_name}")
    else:
        print(f"⚠ ONNX model not found: {onnx_path}")

if len(available_models) == 0:
    print("\n⚠ No models found. Demo will run in limited mode.")
else:
    print(f"\n✓ Loaded {len(available_models)} models for demo")

print("="*60)

Loading available models...
⚠ ONNX model not found: /content/drive/MyDrive/AutoML/deployment/onnx/rank1_resnet18.onnx

⚠ No models found. Demo will run in limited mode.


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import onnxruntime as ort
import gradio as gr
import numpy as np
from PIL import Image
from pathlib import Path
import json
import time
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

# Setup paths
OUTPUT_DIR = Path('/content/drive/MyDrive/AutoML')
MODELS_DIR = OUTPUT_DIR / 'models'
DEPLOYMENT_DIR = OUTPUT_DIR / 'deployment'
ONNX_DIR = DEPLOYMENT_DIR / 'onnx'
RESULTS_DIR = OUTPUT_DIR / 'results'

print(f"Output directory: {OUTPUT_DIR}")

# Create Gradio Interface
def classify_image(image, model_selection):
    """
    Main inference function for Gradio
    """
    if image is None:
        return "Please upload an image", "N/A", None

    if len(available_models) == 0:
        return "No models available", "N/A", None

    try:
        # Get selected model
        if model_selection not in available_models:
            model_selection = list(available_models.keys())[0]

        model = available_models[model_selection]

        # Preprocess image
        processed_image = preprocess_image(image)

        # Run inference
        outputs, inference_time = model.predict(processed_image)

        if outputs is None:
            return "Model inference failed", "N/A", None

        # Get predictions
        predictions = postprocess_outputs(outputs)

        # Format results
        result_text = "Predictions:\n\n"
        for class_name, confidence in predictions.items():
            result_text += f"- {class_name.capitalize()}: {confidence*100:.1f}%\n"

        inference_text = f"Inference Time: {inference_time:.2f} ms"

        return result_text, inference_text, predictions

    except Exception as e:
        return f"Error: {str(e)}", "N/A", None

print("✓ Inference function ready")

✓ All imports successful
Output directory: /content/drive/MyDrive/AutoML
✓ Inference function ready


In [ ]:
def compare_models(image):
    """
    Compare all models on the same image
    """
    if image is None:
        return "Please upload an image"

    if len(available_models) == 0:
        return "No models available for comparison"

    try:
        # Preprocess once
        processed_image = preprocess_image(image)

        comparison_text = "# Model Comparison\n\n"
        comparison_text += "| Model | Top Prediction | Confidence | Inference Time |\n"
        comparison_text += "|-------|----------------|------------|----------------|\n"

        for model_name, model in available_models.items():
            outputs, inference_time = model.predict(processed_image)

            if outputs is not None:
                predictions = postprocess_outputs(outputs)
                top_class = list(predictions.keys())[0]
                top_conf = list(predictions.values())[0]

                comparison_text += f"| {model_name} | {top_class} | {top_conf*100:.1f}% | {inference_time:.2f} ms |\n"

        return comparison_text

    except Exception as e:
        return f"Error during comparison: {str(e)}"

print("✓ Comparison function ready")

✓ Comparison function ready


In [ ]:
# Build Gradio Interface
# Create interface
with gr.Blocks(title="AutoML-Distill Demo", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
     AutoML-Distill: Neural Architecture Search + Knowledge Distillation

    This demo showcases an automated model compression pipeline using:
    - Neural Architecture Search (NAS)to find optimal student architectures
    - Knowledge Distillation to transfer knowledge from teacher to student
    - Multi-objective optimization for accuracy, speed, and model size
    - Deployment optimization with ONNX and quantization

     📊 Key Results:
    - 2-4x model compression with minimal accuracy loss
    - 2-3x inference speedup on CPU
    - Cross-platform deployment with ONNX and quantization

     Upload an image to classify it with optimized models!
    (Note: Models are trained on CIFAR-10, so they work best with images of: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, or trucks)*
    """)

    with gr.Tab("Single Model Inference"):
        gr.Markdown("### Test a single optimized model")

        with gr.Row():
            with gr.Column():
                input_image = gr.Image(type="pil", label="Upload Image")

                if len(available_models) > 0:
                    model_dropdown = gr.Dropdown(
                        choices=list(available_models.keys()),
                        value=list(available_models.keys())[0],
                        label="Select Model"
                    )
                else:
                    model_dropdown = gr.Dropdown(
                        choices=["No models available"],
                        value="No models available",
                        label="Select Model"
                    )

                classify_button = gr.Button("Classify", variant="primary")

            with gr.Column():
                prediction_output = gr.Markdown(label="Predictions")
                inference_time_output = gr.Markdown(label="Performance")
                prediction_chart = gr.Label(label="Confidence Scores")

        classify_button.click(
            fn=classify_image,
            inputs=[input_image, model_dropdown],
            outputs=[prediction_output, inference_time_output, prediction_chart]
        )

    with gr.Tab("Model Comparison"):
        gr.Markdown(" Compare all models on the same image")

        with gr.Row():
            with gr.Column():
                compare_input_image = gr.Image(type="pil", label="Upload Image")
                compare_button = gr.Button("Compare All Models", variant="primary")

            with gr.Column():
                comparison_output = gr.Markdown(label="Comparison Results")

        compare_button.click(
            fn=compare_models,
            inputs=[compare_input_image],
            outputs=[comparison_output]
        )

    with gr.Tab("About"):
        gr.Markdown("""
         📖 About This Project

        Pipeline Overview:

        1. Model Profiling: Analyze baseline models (parameters, FLOPs, latency)
        2. Knowledge Distillation: Experiment with different distillation strategies
        3. Search Space Design: Define architecture and hyperparameter spaces
        4. Neural Architecture Search: Evolutionary algorithm finds optimal configs
        5. Full Training: Train best configurations on complete dataset
        6. Deployment Optimization: ONNX conversion and quantization
        7. Demo Interface: This interactive showcase!

        Technologies Used:
        - PyTorch for model training
        - ONNX for cross-platform deployment
        - Gradio for web interface
        - Evolutionary Algorithms for NAS

        Key Features:
        - Automated architecture search
        - Multi-objective optimization
        - Knowledge distillation
        - Hardware-aware optimization
        - Production-ready deployment formats

         Results:
        - 2-4x smaller models
        - 2-3x faster inference
        - <2% accuracy loss
        - Cross-platform compatible

        ---

        Built with: PyTorch • ONNX • Gradio • NumPy

        Project Structure: 7 Jupyter notebooks covering the complete ML pipeline
        """)

    gr.Markdown("""
    --- General Information:
    💡 Tip: For best results, upload images similar to CIFAR-10 classes (vehicles, animals, etc.)
    """)

print("✓ Gradio interface built successfully")

✓ Gradio interface built successfully


In [ ]:
# launch Demo
print("\n" + "="*80)
print("LAUNCHING DEMO")
print("="*80)
print("\n🚀 Starting Gradio interface...")
print("\n📝 Available models:")
for model_name in available_models.keys():
    print(f"  ✓ {model_name}")

if len(available_models) == 0:
    print("\n⚠ Warning: No models loaded. Demo will run in limited mode.")
    print("   Make sure to run Notebooks 5 & 6 first to generate models.")

print("\n" + "="*80)
print("Demo will launch below. Share the public link to showcase your work!")
print("="*80)

# Launch with share=True to get public URL
demo.launch(
    share=True,  # Creates public URL
    debug=True,
    show_error=True
)


LAUNCHING DEMO

🚀 Starting Gradio interface...

📝 Available models:

⚠ Warning: No models loaded. Demo will run in limited mode.
   Make sure to run Notebooks 5 & 6 first to generate models.

Demo will launch below. Share the public link to showcase your work!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://da167199fd80f963b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://da167199fd80f963b7.gradio.live
